# XGBoost

In [1]:
from sklearn.model_selection import KFold
from IPython.display import display
from ipywidgets import IntProgress
import xgboost as xgb
import pandas as pd
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [3]:
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))

In [4]:
calcular_auc = '"{}"'.format(os.path.join(current_folder, '..', 'Calcular AUC.ipynb'))

In [5]:
set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))

In [6]:
predicciones_csv = os.path.join(current_folder, '..', 'predictions.csv')

In [7]:
resultados_csv = os.path.join(current_folder, '..', 'resultados.csv')

Cargo el df con los features.

In [8]:
pd.options.mode.chained_assignment = None

In [9]:
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [10]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

Cargo los sets de entrenamiento, testing y predicción.

In [32]:
%run $set_de_entrenamiento_testing_y_prediccion

## Entrenamiento rápido

In [33]:
training = labels_training.merge(df_features, how='inner', on='person')

In [34]:
training.head()

,label,screen_resolution_height,screen_resolution_width,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,Wrexham,days until 31-05 mean,days until 31-05 std,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero,compras Febrero,compras Marzo,compras Abril,compras Mayo,dias ultima compra
person,,,,,,,,,,,,,,,,,,,,,,,,,
0566e9c1,0,568.000000,320.000000,6.0,3.0,1.0,1.0,15.0,0.0,1.0,0.0,1.0,...,0.0,7.250000,2.281496,0.0,17.0,True,0,0,0,0,1,9
6ec7ee77,0,640.000000,360.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,3.000000,0.000000,0.0,1.0,True,0,0,0,0,0,152
87ed62de,0,864.000000,1152.000000,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,12.941176,0.242536,1.0,0.0,True,0,0,0,0,0,152
db2c4d27,1,640.491228,429.140351,3.0,31.0,3.0,1.0,166.0,0.0,92.0,13.0,20.0,...,0.0,54.781915,15.926238,7.0,107.0,True,0,0,1,0,0,71
be65035b,0,640.000000,360.000000,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,...,0.0,55.000000,0.755929,0.0,3.0,True,0,0,0,0,0,152


In [35]:
columns = list(training.columns)
columns.remove('label')

In [36]:
dtrain = xgb.DMatrix(training.loc[:, columns], label=training['label'])

Hiperparámetros:

In [37]:
param = {'max_depth': 4, 'eta': 0.54, 'gamma': 7, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

In [38]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

Predigo:

In [39]:
labels_test = labels_test.merge(df_features, how='inner', on='person')

In [40]:
test_matrix = xgb.DMatrix(labels_test.loc[:, columns])

In [41]:
labels_test['label_predicted'] = bst.predict(test_matrix)

Métrica de la predicción:

In [42]:
%run $calcular_auc

auc score: 0.8538769902160098


## Búsqueda de hiperparámetros con k-fold

Vamos a hacer un kfold con sklearn.

In [134]:
splits = 2
max_depth_values = 3
eta_values = 4
gamma_values = 3
num_round = 1

In [135]:
param = {
    'max_depth': 0,
    'eta': 0,
    'gamma': 0,
    'silent': 1,
    'objective': 'binary:logistic',
    'nthread': 10,
    'eval_metric': 'auc'
}

In [136]:
labels_with_features = labels.merge(df_features, how='inner', on='person')

In [137]:
columns = list(labels_with_features.columns)
columns.remove('label')

In [138]:
f = IntProgress(min=0, max=splits*max_depth_values*eta_values*gamma_values)
display(f) # display the bar

kf = KFold(n_splits=splits, shuffle=False)
results = pd.DataFrame(columns=['k', 'max_depth', 'eta', 'gamma', 'auc'])
index = 0
k = 0
for train_index, test_index in kf.split(labels):
    
    labels_training = labels_with_features.iloc[train_index]
    labels_test = labels_with_features.iloc[test_index]
    assert(labels_training.merge(labels_test, how='inner', on='person').shape[0] == 0)
    train_matrix = xgb.DMatrix(labels_training.loc[:, columns], label=labels_training['label'])
    test_matrix = xgb.DMatrix(labels_test.loc[:, columns])
    
    for max_depth, eta, gamma in np.ndindex((max_depth_values, eta_values, gamma_values)):
        eta=eta/eta_values
        param['max_depth'] = max_depth
        param['eta'] = eta
        param['gamma'] = gamma
        
        bst = xgb.train(param, train_matrix, num_round)
        labels_test['label_predicted'] = bst.predict(test_matrix)
        
        results.loc[index] = k, max_depth, eta, gamma, calculate_auc()
        
        index+=1
        f.value += 1
    
    k += 1

IntProgress(value=0, max=72)

Promedio los hiperparámetros.

In [139]:
results_mean = results.groupby(['max_depth', 'eta', 'gamma'])[['auc']].mean()

In [140]:
mejor_resultado = results_mean.loc[results_mean.idxmax()]
mejor_resultado

,,,auc
max_depth,eta,gamma,
2.0,0.25,0.0,0.747723


Escribo los nuevos resultados en un archivo.

In [43]:
mejores_resultados = pd.read_csv(resultados_csv, index_col='fecha')

In [44]:
mejores_resultados

,max_depth,eta,gamma,k,num_round,auc,features
fecha,,,,,,,
2018-11-11 10:25,4.0,0.6,6.0,10,10,0.859342,"screen_resolution_height,screen_resolution_wid..."


In [142]:
mejor_resultado['k'] = splits
mejor_resultado['num_round'] = num_round
mejor_resultado['features'] = ','.join(columns)
mejor_resultado['fecha'] = pd.datetime.now().strftime("%Y-%m-%d %H:%M")
mejor_resultado.reset_index(inplace=True)
mejor_resultado = mejor_resultado[['fecha', 'max_depth', 'eta', 'gamma', 'k', 'num_round', 'features']]
mejor_resultado.set_index('fecha', inplace=True)

In [143]:
mejores_resultados = mejores_resultados.append(mejor_resultado)

In [144]:
mejores_resultados.to_csv(resultados_csv)

## Predecir labels desconocidos

In [45]:
training = labels.merge(df_features, how='inner', on='person')

In [46]:
columns = list(training.columns)
columns.remove('label')

In [47]:
dtrain = xgb.DMatrix(training.loc[:, columns], label=training['label'])

In [48]:
param['max_depth'] = 4
param['eta'] = 0.6
param['gamma'] = 6

In [64]:
num_round = 100
bst = xgb.train(param, dtrain, num_round)

Predigo:

In [65]:
labels_to_predict_with_features = labels_to_predict.merge(df_features, how='inner', on='person')

In [66]:
assert(labels_to_predict.shape[0] == labels_to_predict_with_features.shape[0])

In [67]:
matrix = xgb.DMatrix(labels_to_predict_with_features.loc[:, columns])

In [68]:
labels_to_predict['label'] = bst.predict(matrix)

In [69]:
labels_to_predict.to_csv(predicciones_csv)